# 1. netkeiba.comからのスクレイピング

In [1]:
import numpy as np
import pandas as pd
import time
from tqdm import tqdm as tqdm

In [6]:
import pandas as pd

url = 'https://db.netkeiba.com/race/201901010101' # 2019 開催年/ 01 開催場所/ 01 1回目/ 01 1日目/ 01 ラウンド
pd.read_html(url)[0] # list で返ってくるものの要素が[0]を選択 = データフレームにかわる

pandas.core.frame.DataFrame

開催場所 = {01:札幌, 02:函館, 03:福島, 04:新潟, 05:東京, 06:中山, 07:中京, 08:京都, 09:阪神, 10:小倉}

In [2]:
import time
from tqdm import tqdm as tqdm # pip install tqdm

def scrape_race_results(race_id_list, pre_race_results={}): # 競馬サイトのスクレイピング関数の作成
    """
        受け取った'race_id_list'の競馬サイトからスクレイピングを行いそれぞれのレース結果のデータフレームが入った辞書型の変数を返す
    """
    race_results = pre_race_results # race_results を辞書型として作成
    for race_id in tqdm(race_id_list):
        if race_id in race_results.keys():
            continue
        try: # 途中でWiFiなど切れてfor文が終わってしまったとした時に、終わった分を送ってくれるようにtry文を作成
            url = 'https://db.netkeiba.com/race/' + race_id
            race_results[race_id] = pd.read_html(url)[0] # 辞書型のkeyを指定して、辞書型の要素に pd.read_html で取得したデータフレームを代入
            time.sleep(1) # サーバーへの負担軽減
        except IndexError: # 存在しないデータへアクセスした際のエラー処理
            continue
        except:
            break
    return race_results

In [3]:
race_id_list = []
for place in range(1, 11, 1): # race_id_listの作成
    for kai in range(1, 6, 1):
        for day in range(1, 9, 1):
            for r in range(1, 13, 1):
                race_id = '2019' + str(place).zfill(2) + str(kai).zfill(2) + str(day).zfill(2) + str(r).zfill(2) # zfill(2) : 二桁に揃える。1 の場合は 01 になる
                race_id_list.append(race_id)

In [4]:
len(race_id_list)

4800

In [14]:
test = scrape_race_results(race_id_list)

100%|██████████| 4800/4800 [27:20<00:00,  2.93it/s]


In [22]:
test['201901010101']

,着順,枠番,馬番,馬名,性齢,斤量,騎手,タイム,着差,単勝,人気,馬体重,調教師
0,1,1,1,ゴルコンダ,牡2,54,ルメール,1:48.3,NaN,1.4,1,518(-16),[東] 木村哲也
1,2,3,3,プントファイヤー,牡2,54,岩田康誠,1:50.1,大,3.5,2,496(-8),[東] 手塚貴久
2,3,4,4,ラグリマスネグラス,牡2,51,団野大成,1:50.9,5,46.6,6,546(+6),[東] 藤沢和雄
3,4,8,9,キタノコドウ,牡2,51,菅原明良,1:51.5,3.1/2,56.8,7,458(-8),[東] 高木登
4,5,5,5,ネモフィラブルー,牡2,54,川島信二,1:51.7,1.1/2,140.3,9,436(0),[西] 矢作芳人
5,6,8,8,マイネルラクスマン,牡2,54,丹内祐次,1:52.1,2.1/2,9.7,3,480(+8),[東] 金成貴史
6,7,2,2,サンモンテベロ,牝2,54,黛弘人,1:52.5,2.1/2,114.7,8,450(+2),[東] 中野栄治
7,8,7,7,エスカレーション,牝2,54,藤岡佑介,1:52.5,アタマ,26.1,5,448(-4),[東] 高柳瑞樹
8,9,6,6,セイウンジュリア,牝2,54,池添謙一,1:52.6,クビ,16.4,4,470(0),[西] 浅見秀一


In [23]:
for key in test.keys(): # race_idごとのデータフレームのindexをそれぞれのrace_idに置き換える処理
    test[key].index = [key] * len(test[key])

In [24]:
test

{'201901010101':               着順  枠番  馬番         馬名  性齢  斤量    騎手     タイム     着差     単勝  人気  \
 201901010101   1   1   1      ゴルコンダ  牡2  54  ルメール  1:48.3    NaN    1.4   1   
 201901010101   2   3   3   プントファイヤー  牡2  54  岩田康誠  1:50.1      大    3.5   2   
 201901010101   3   4   4  ラグリマスネグラス  牡2  51  団野大成  1:50.9      5   46.6   6   
 201901010101   4   8   9     キタノコドウ  牡2  51  菅原明良  1:51.5  3.1/2   56.8   7   
 201901010101   5   5   5   ネモフィラブルー  牡2  54  川島信二  1:51.7  1.1/2  140.3   9   
 201901010101   6   8   8  マイネルラクスマン  牡2  54  丹内祐次  1:52.1  2.1/2    9.7   3   
 201901010101   7   2   2    サンモンテベロ  牝2  54   黛弘人  1:52.5  2.1/2  114.7   8   
 201901010101   8   7   7   エスカレーション  牝2  54  藤岡佑介  1:52.5    アタマ   26.1   5   
 201901010101   9   6   6   セイウンジュリア  牝2  54  池添謙一  1:52.6     クビ   16.4   4   
 
                    馬体重       調教師  
 201901010101  518(-16)  [東] 木村哲也  
 201901010101   496(-8)  [東] 手塚貴久  
 201901010101   546(+6)  [東] 藤沢和雄  
 201901010101   458(-8)   [東] 高木登  
 2

In [25]:
results = pd.concat([test[key] for key in test.keys()], sort=False) #resultsにurlから取ってきた全てのデータフレームを縦に結合 pd.concat([df1, df2]) / リスト内表記 [i**2 for i in range(5)]

In [26]:
results

,着順,枠番,馬番,馬名,性齢,斤量,騎手,タイム,着差,単勝,人気,馬体重,調教師
201901010101,1,1,1,ゴルコンダ,牡2,54.0,ルメール,1:48.3,NaN,1.4,1.0,518(-16),[東] 木村哲也
201901010101,2,3,3,プントファイヤー,牡2,54.0,岩田康誠,1:50.1,大,3.5,2.0,496(-8),[東] 手塚貴久
201901010101,3,4,4,ラグリマスネグラス,牡2,51.0,団野大成,1:50.9,5,46.6,6.0,546(+6),[東] 藤沢和雄
201901010101,4,8,9,キタノコドウ,牡2,51.0,菅原明良,1:51.5,3.1/2,56.8,7.0,458(-8),[東] 高木登
201901010101,5,5,5,ネモフィラブルー,牡2,54.0,川島信二,1:51.7,1.1/2,140.3,9.0,436(0),[西] 矢作芳人
...,...,...,...,...,...,...,...,...,...,...,...,...,...
201910020812,10,7,12,オーシャンスケイプ,牝3,52.0,大野拓弥,1:49.8,3/4,45.7,10.0,414(+6),[西] 西村真幸
201910020812,11,3,3,プリヴェット,牝4,55.0,藤岡康太,1:50.3,3,29.8,9.0,486(+2),[西] 今野貞一
201910020812,12,5,7,アドマイヤクィーン,牝4,55.0,松山弘平,1:50.3,クビ,29.6,8.0,476(+6),[西] 須貝尚介
201910020812,13,8,13,タムロドリーム,牝3,49.0,亀田温心,1:50.4,1/2,134.8,12.0,472(+2),[西] 西園正都


In [27]:
results.to_pickle('results.pickle') #pickle形式で保存

In [28]:
pd.read_pickle('results.pickle') #pickle形式ファイルの読み込み

,着順,枠番,馬番,馬名,性齢,斤量,騎手,タイム,着差,単勝,人気,馬体重,調教師
201901010101,1,1,1,ゴルコンダ,牡2,54.0,ルメール,1:48.3,NaN,1.4,1.0,518(-16),[東] 木村哲也
201901010101,2,3,3,プントファイヤー,牡2,54.0,岩田康誠,1:50.1,大,3.5,2.0,496(-8),[東] 手塚貴久
201901010101,3,4,4,ラグリマスネグラス,牡2,51.0,団野大成,1:50.9,5,46.6,6.0,546(+6),[東] 藤沢和雄
201901010101,4,8,9,キタノコドウ,牡2,51.0,菅原明良,1:51.5,3.1/2,56.8,7.0,458(-8),[東] 高木登
201901010101,5,5,5,ネモフィラブルー,牡2,54.0,川島信二,1:51.7,1.1/2,140.3,9.0,436(0),[西] 矢作芳人
...,...,...,...,...,...,...,...,...,...,...,...,...,...
201910020812,10,7,12,オーシャンスケイプ,牝3,52.0,大野拓弥,1:49.8,3/4,45.7,10.0,414(+6),[西] 西村真幸
201910020812,11,3,3,プリヴェット,牝4,55.0,藤岡康太,1:50.3,3,29.8,9.0,486(+2),[西] 今野貞一
201910020812,12,5,7,アドマイヤクィーン,牝4,55.0,松山弘平,1:50.3,クビ,29.6,8.0,476(+6),[西] 須貝尚介
201910020812,13,8,13,タムロドリーム,牝3,49.0,亀田温心,1:50.4,1/2,134.8,12.0,472(+2),[西] 西園正都


# 2 データの整形・前処理

In [90]:
import pandas as pd

results = pd.read_pickle('results.pickle')

In [91]:
results

,着順,枠番,馬番,馬名,性齢,斤量,騎手,タイム,着差,単勝,人気,馬体重,調教師
201901010101,1,1,1,ゴルコンダ,牡2,54.0,ルメール,1:48.3,NaN,1.4,1.0,518(-16),[東] 木村哲也
201901010101,2,3,3,プントファイヤー,牡2,54.0,岩田康誠,1:50.1,大,3.5,2.0,496(-8),[東] 手塚貴久
201901010101,3,4,4,ラグリマスネグラス,牡2,51.0,団野大成,1:50.9,5,46.6,6.0,546(+6),[東] 藤沢和雄
201901010101,4,8,9,キタノコドウ,牡2,51.0,菅原明良,1:51.5,3.1/2,56.8,7.0,458(-8),[東] 高木登
201901010101,5,5,5,ネモフィラブルー,牡2,54.0,川島信二,1:51.7,1.1/2,140.3,9.0,436(0),[西] 矢作芳人
...,...,...,...,...,...,...,...,...,...,...,...,...,...
201910020812,10,7,12,オーシャンスケイプ,牝3,52.0,大野拓弥,1:49.8,3/4,45.7,10.0,414(+6),[西] 西村真幸
201910020812,11,3,3,プリヴェット,牝4,55.0,藤岡康太,1:50.3,3,29.8,9.0,486(+2),[西] 今野貞一
201910020812,12,5,7,アドマイヤクィーン,牝4,55.0,松山弘平,1:50.3,クビ,29.6,8.0,476(+6),[西] 須貝尚介
201910020812,13,8,13,タムロドリーム,牝3,49.0,亀田温心,1:50.4,1/2,134.8,12.0,472(+2),[西] 西園正都


In [101]:
results.columns

Index(['着順', '枠番', '馬番', '馬名', '性齢', '斤量', '騎手', 'タイム', '着差', '単勝', '人気',
       '馬体重', '調教師'],
      dtype='object')

In [102]:
results['着順'].value_counts()

4       2864
1       2863
3       2859
2       2857
5       2855
6       2853
7       2836
8       2801
9       2702
10      2553
11      2372
12      2161
13      1938
14      1702
15      1403
16      1101
4        330
1        330
3        329
6        329
5        328
2        328
7        327
8        322
9        307
10       283
11       263
12       229
中        209
17       198
13       197
18       165
14       157
除        157
15       120
取         57
16        26
17        22
2(降)       2
3(降)       1
5(降)       1
18         1
Name: 着順, dtype: int64

In [103]:
results['着順'].astype(str).value_counts() # 4,1,3,6,5,2,7,8,9,10,... これらはおそらく数字と文字列によって分けられてしまっているので文字列で統一

4       3194
1       3193
3       3188
2       3185
5       3183
6       3182
7       3163
8       3123
9       3009
10      2836
11      2635
12      2390
13      2135
14      1859
15      1523
16      1127
17       220
中        209
18       166
除        157
取         57
2(降)       2
5(降)       1
3(降)       1
Name: 着順, dtype: int64

In [104]:
results[results['着順'].astype(str).str.contains('\D')]['着順'].value_counts()

中       209
除       157
取        57
2(降)      2
5(降)      1
3(降)      1
Name: 着順, dtype: int64

In [105]:
results2 = results[~results['着順'].astype(str).str.contains('\D')] # ~:not 文字が含まれているもの以外を取得しresults2に代入 

In [106]:
results2['性齢']

201901010101    牡2
201901010101    牡2
201901010101    牡2
201901010101    牡2
201901010101    牡2
                ..
201910020812    牝3
201910020812    牝4
201910020812    牝4
201910020812    牝3
201910020812    牝3
Name: 性齢, Length: 43311, dtype: object

In [107]:
results2['性'] = results2['性齢'].map(lambda x: str(x)[0])

<ipython-input-107-59c4ff887a8d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results2['性'] = results2['性齢'].map(lambda x: str(x)[0])


In [108]:
results2['年齢'] = results2['性齢'].map(lambda x: str(x)[1:])

<ipython-input-108-4d3e75abd698>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results2['年齢'] = results2['性齢'].map(lambda x: str(x)[1:])


In [109]:
results2['馬体重']

201901010101    518(-16)
201901010101     496(-8)
201901010101     546(+6)
201901010101     458(-8)
201901010101      436(0)
                  ...   
201910020812     414(+6)
201910020812     486(+2)
201910020812     476(+6)
201910020812     472(+2)
201910020812     458(-6)
Name: 馬体重, Length: 43311, dtype: object

In [38]:
results2['体重'] = results2['馬体重'].str.split('(', expand=True)[0].astype(int) # ディフォルトでは expand = False だが、True にすると複数列に分割することができる

<ipython-input-38-1cb3c3c12f33>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results2['体重'] = results2['馬体重'].str.split('(', expand=True)[0].astype(int)


In [39]:
results2['体重変化'] = results2['馬体重'].str.split('(', expand=True)[1].str[:-1].astype(int)  # -1:最後, [:,-1]最初から最後でけ含まないで取得

<ipython-input-39-0eca72f641df>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results2['体重変化'] = results2['馬体重'].str.split('(', expand=True)[1].str[:-1].astype(int)


In [40]:
results2

,着順,枠番,馬番,馬名,性齢,斤量,騎手,タイム,着差,単勝,人気,馬体重,調教師,性,年齢,体重,体重変化
201901010101,1,1,1,ゴルコンダ,牡2,54.0,ルメール,1:48.3,NaN,1.4,1.0,518(-16),[東] 木村哲也,牡,2,518,-16
201901010101,2,3,3,プントファイヤー,牡2,54.0,岩田康誠,1:50.1,大,3.5,2.0,496(-8),[東] 手塚貴久,牡,2,496,-8
201901010101,3,4,4,ラグリマスネグラス,牡2,51.0,団野大成,1:50.9,5,46.6,6.0,546(+6),[東] 藤沢和雄,牡,2,546,6
201901010101,4,8,9,キタノコドウ,牡2,51.0,菅原明良,1:51.5,3.1/2,56.8,7.0,458(-8),[東] 高木登,牡,2,458,-8
201901010101,5,5,5,ネモフィラブルー,牡2,54.0,川島信二,1:51.7,1.1/2,140.3,9.0,436(0),[西] 矢作芳人,牡,2,436,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201910020812,10,7,12,オーシャンスケイプ,牝3,52.0,大野拓弥,1:49.8,3/4,45.7,10.0,414(+6),[西] 西村真幸,牝,3,414,6
201910020812,11,3,3,プリヴェット,牝4,55.0,藤岡康太,1:50.3,3,29.8,9.0,486(+2),[西] 今野貞一,牝,4,486,2
201910020812,12,5,7,アドマイヤクィーン,牝4,55.0,松山弘平,1:50.3,クビ,29.6,8.0,476(+6),[西] 須貝尚介,牝,4,476,6
201910020812,13,8,13,タムロドリーム,牝3,49.0,亀田温心,1:50.4,1/2,134.8,12.0,472(+2),[西] 西園正都,牝,3,472,2


In [41]:
results2.drop(['タイム', '着差', '調教師'], axis=1, inplace=True)

/Applications/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:3990: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [42]:
results2

,着順,枠番,馬番,馬名,性齢,斤量,騎手,単勝,人気,馬体重,性,年齢,体重,体重変化
201901010101,1,1,1,ゴルコンダ,牡2,54.0,ルメール,1.4,1.0,518(-16),牡,2,518,-16
201901010101,2,3,3,プントファイヤー,牡2,54.0,岩田康誠,3.5,2.0,496(-8),牡,2,496,-8
201901010101,3,4,4,ラグリマスネグラス,牡2,51.0,団野大成,46.6,6.0,546(+6),牡,2,546,6
201901010101,4,8,9,キタノコドウ,牡2,51.0,菅原明良,56.8,7.0,458(-8),牡,2,458,-8
201901010101,5,5,5,ネモフィラブルー,牡2,54.0,川島信二,140.3,9.0,436(0),牡,2,436,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201910020812,10,7,12,オーシャンスケイプ,牝3,52.0,大野拓弥,45.7,10.0,414(+6),牝,3,414,6
201910020812,11,3,3,プリヴェット,牝4,55.0,藤岡康太,29.8,9.0,486(+2),牝,4,486,2
201910020812,12,5,7,アドマイヤクィーン,牝4,55.0,松山弘平,29.6,8.0,476(+6),牝,4,476,6
201910020812,13,8,13,タムロドリーム,牝3,49.0,亀田温心,134.8,12.0,472(+2),牝,3,472,2


In [43]:
results2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 43311 entries, 201901010101 to 201910020812
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   着順      43311 non-null  object 
 1   枠番      43311 non-null  int64  
 2   馬番      43311 non-null  int64  
 3   馬名      43311 non-null  object 
 4   性齢      43311 non-null  object 
 5   斤量      43311 non-null  float64
 6   騎手      43311 non-null  object 
 7   単勝      43311 non-null  object 
 8   人気      43311 non-null  float64
 9   馬体重     43311 non-null  object 
 10  性       43311 non-null  object 
 11  年齢      43311 non-null  object 
 12  体重      43311 non-null  int64  
 13  体重変化    43311 non-null  int64  
dtypes: float64(2), int64(4), object(8)
memory usage: 6.2+ MB


In [44]:
results2['着順'] = results2['着順'].astype(int)

<ipython-input-44-40c51b14d53c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results2['着順'] = results2['着順'].astype(int)


In [45]:
results2['単勝'] = results2['単勝'].astype(float)

<ipython-input-45-df3b86fb780a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results2['単勝'] = results2['単勝'].astype(float)


In [46]:
results2.drop(['性齢', '馬体重'], axis=1, inplace=True)

/Applications/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:3990: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [47]:
results2['年齢'] = results2['年齢'].astype(int)

<ipython-input-47-4a8cd1c034f5>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results2['年齢'] = results2['年齢'].astype(int)


In [48]:
results2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 43311 entries, 201901010101 to 201910020812
Data columns (total 12 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   着順      43311 non-null  int64  
 1   枠番      43311 non-null  int64  
 2   馬番      43311 non-null  int64  
 3   馬名      43311 non-null  object 
 4   斤量      43311 non-null  float64
 5   騎手      43311 non-null  object 
 6   単勝      43311 non-null  float64
 7   人気      43311 non-null  float64
 8   性       43311 non-null  object 
 9   年齢      43311 non-null  int64  
 10  体重      43311 non-null  int64  
 11  体重変化    43311 non-null  int64  
dtypes: float64(3), int64(6), object(3)
memory usage: 5.5+ MB


In [129]:
import datetime

def preprocessing(results):
    """
        引数として受け取ったデータの前処理
    """
    df = results.copy()

    if '着順' in df.columns:
        # 着順に数字以外の文字列が含まれているものを取り除く
        df = df[~df['着順'].astype(str).str.contains('\D')]
        df['着順'] = df['着順'].astype(int)

    if '性齢' in df.columns:
        # 性齢を性と年齢に分ける
        df['性'] = df['性齢'].map(lambda x: str(x)[0])
        df['年齢'] = df['性齢'].map(lambda x: str(x)[1:]).astype(int)

    if '馬体重' in df.columns:
        # 馬体重を体重と体重変化に分ける
        df['体重'] = df['馬体重'].str.split('(', expand=True)[0].astype(int)
        df['体重変化'] = df['馬体重'].str.split('(', expand=True)[1].str[:-1].astype(int)

    if '単勝' in df.columns:
        # データをint, floatに変換
        df['単勝'] = df['単勝'].astype(float)

    for drop_col in ['タイム', '着差', '調教師', '性齢', '馬体重']:
        if drop_col in df.columns:
            # 不要な列の削除
            df.drop([drop_col], axis=1, inplace=True)

    if 'date' in df.columns:
        # 開催日時の処理(時間データに変更)
        df['date'] = pd.to_datetime(df['date'], format='%Y年%m月%d日')

    return df

In [50]:
results2 = preprocessing(results)

In [51]:
results.info()

<class 'pandas.core.frame.DataFrame'>
Index: 43738 entries, 201901010101 to 201910020812
Data columns (total 13 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   着順      43738 non-null  object 
 1   枠番      43738 non-null  int64  
 2   馬番      43738 non-null  int64  
 3   馬名      43738 non-null  object 
 4   性齢      43738 non-null  object 
 5   斤量      43738 non-null  float64
 6   騎手      43738 non-null  object 
 7   タイム     43315 non-null  object 
 8   着差      40125 non-null  object 
 9   単勝      43738 non-null  object 
 10  人気      43524 non-null  float64
 11  馬体重     43738 non-null  object 
 12  調教師     43738 non-null  object 
dtypes: float64(2), int64(2), object(9)
memory usage: 4.7+ MB


In [52]:
results2.to_pickle('results2.pickle')

# 3. ロジスティック回帰で競馬予測

In [7]:
results = pd.read_pickle('results2.pickle')
results

,着順,枠番,馬番,馬名,斤量,騎手,単勝,人気,性,年齢,体重,体重変化
201901010101,1,1,1,ゴルコンダ,54.0,ルメール,1.4,1.0,牡,2,518,-16
201901010101,2,3,3,プントファイヤー,54.0,岩田康誠,3.5,2.0,牡,2,496,-8
201901010101,3,4,4,ラグリマスネグラス,51.0,団野大成,46.6,6.0,牡,2,546,6
201901010101,4,8,9,キタノコドウ,51.0,菅原明良,56.8,7.0,牡,2,458,-8
201901010101,5,5,5,ネモフィラブルー,54.0,川島信二,140.3,9.0,牡,2,436,0
...,...,...,...,...,...,...,...,...,...,...,...,...
201910020812,10,7,12,オーシャンスケイプ,52.0,大野拓弥,45.7,10.0,牝,3,414,6
201910020812,11,3,3,プリヴェット,55.0,藤岡康太,29.8,9.0,牝,4,486,2
201910020812,12,5,7,アドマイヤクィーン,55.0,松山弘平,29.6,8.0,牝,4,476,6
201910020812,13,8,13,タムロドリーム,49.0,亀田温心,134.8,12.0,牝,3,472,2


In [8]:
#今回は分類問題として解く。分類内容は1位,2位,3位,4位以下の4つの分類とする。
#そのため、'rank'列で着順を仕分ける
clip_rank = lambda x: x if x<4 else 4

results['rank'] = results['着順'].map(clip_rank)

In [9]:
results['rank'].value_counts()

4    33745
1     3193
3     3188
2     3185
Name: rank, dtype: int64

In [10]:
results['馬名'].value_counts() #カテゴリ変数をダミー変数化する必要があるが、馬名を使ってしまうとすごいデータ量が多くなってしまう（スパース問題）ため今回は使わない

ラタンドレス       16
ワタシヲマッテル     15
イチザティアラ      15
トゥーフラッシー     15
ジューンバラード     14
             ..
クインズモアナ       1
ゲンセキ          1
ナススイート        1
ウイングハーモニー     1
シリウスインパクト     1
Name: 馬名, Length: 11345, dtype: int64

In [11]:
results['騎手'].value_counts() #200件くらいであればダミー変数化しても問題ない

和田竜二    822
三浦皇成    804
松山弘平    802
岩田康誠    776
大野拓弥    709
       ... 
高松亮       1
石堂響       1
山田祥雄      1
阪野学       1
永島太郎      1
Name: 騎手, Length: 187, dtype: int64

In [12]:
results['性'].value_counts()

牡    23678
牝    17646
セ     1987
Name: 性, dtype: int64

In [13]:
results.drop(['着順', '馬名'], axis=1, inplace=True)

In [14]:
# ワンホットエンコーディング：列を増やすエンコーディング
# 順序に意味がない名義尺度では、ラベルエンコーディングで変換される整数値を用いて回帰分析を行うことはできない。
# ラベルエンコーディングで1つの変数の離散値として扱う場合は、決定木分析、ランダムフォレストなどの分析と相性が良い。また、ワンホットエンコーディングよりも変数の数を節約できる。
# pandas.DataFrameの場合は、デフォルトではデータ型dtypeがobject（おもに文字列）またはcategoryである列がすべてダミー変数化される。
# dummy_na=True : デフォルトでは欠損値は除外して処理される（全ての項目が0になる）が、Trueにすると欠損値も一つのカテゴリーとしてダミー変数化される
# drop_first=True : k個のカテゴリーをk-1でダミー化する（多重共線性を回避するため）
results_d = pd.get_dummies(results)

In [15]:
results_d

,枠番,馬番,斤量,単勝,人気,年齢,体重,体重変化,rank,騎手_アヴドゥ,...,騎手_高田潤,騎手_高野和馬,騎手_鮫島克駿,騎手_鮫島良太,騎手_黒岩悠,騎手_黛弘人,騎手_Ｍ．デム,性_セ,性_牝,性_牡
201901010101,1,1,54.0,1.4,1.0,2,518,-16,1,0,...,0,0,0,0,0,0,0,0,0,1
201901010101,3,3,54.0,3.5,2.0,2,496,-8,2,0,...,0,0,0,0,0,0,0,0,0,1
201901010101,4,4,51.0,46.6,6.0,2,546,6,3,0,...,0,0,0,0,0,0,0,0,0,1
201901010101,8,9,51.0,56.8,7.0,2,458,-8,4,0,...,0,0,0,0,0,0,0,0,0,1
201901010101,5,5,54.0,140.3,9.0,2,436,0,4,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201910020812,7,12,52.0,45.7,10.0,3,414,6,4,0,...,0,0,0,0,0,0,0,0,1,0
201910020812,3,3,55.0,29.8,9.0,4,486,2,4,0,...,0,0,0,0,0,0,0,0,1,0
201910020812,5,7,55.0,29.6,8.0,4,476,6,4,0,...,0,0,0,0,0,0,0,0,1,0
201910020812,8,13,49.0,134.8,12.0,3,472,2,4,0,...,0,0,0,0,0,0,0,0,1,0


In [16]:
results_d.to_pickle('results3.pickle')

In [62]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
#from sklearn.preprocessing import StandardScaler

X=results_d.drop(['rank'], axis=1)
y=results_d['rank']

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.3, random_state=71) #訓練データとテストデータに分割（stratify:訓練データとテストデータの目的変数の散らばり度合いを均一化してくれる）

In [63]:
model = LogisticRegression()
model.fit(X_train, y_train)

print(f'決定係数(X_train) : {model.score(X_train, y_train):.4f},\n決定係数(X_test) : {model.score(X_test, y_test):.4f}') #model.score : 決定係数

決定係数(X_train) : 0.7791,
決定係数(X_test) : 0.7791


/Applications/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [64]:
y_pred = model.predict(X_test)

In [65]:
y_pred

array([4, 4, 4, ..., 4, 4, 4])

In [66]:
pd.Series(y_pred).value_counts() #不均衡データによる誤予測（今回のデータでは 4:4位以下 のデータが1,2,3位のデータよりとても多くあるので4と予測しておけば正解だよねという形に隔たった予測になっている） → アンダーサンプリング
#pd.DataFrame(y_pred).iloc[:,0].value_counts()

4    12994
dtype: int64

In [67]:
y_train.value_counts()

4    23621
1     2235
3     2232
2     2229
Name: rank, dtype: int64

In [68]:
# アンダーサンプリング
# 訓練データの目的変数の1,2,3,4のそれぞれの割合を均一に変更、データ数をそれぞれ約2300個の計9200個の訓練データを作成
from imblearn.under_sampling import RandomUnderSampler #pip install -U imbalanced-learn

rank_clip = []
for rank in range(1,5):
    rank_clip.append(y_train.value_counts()[rank] if y_train.value_counts()[rank]< y_train.value_counts()[1]*2 else y_train.value_counts()[1])

rank_clip

rus = RandomUnderSampler(sampling_strategy={1:rank_clip[0], 2:rank_clip[1], 3:rank_clip[2], 4:rank_clip[3]}, random_state=71)
X_train_rus, y_train_rus = rus.fit_resample(X_train, y_train)

In [69]:
pd.Series(y_train_rus).value_counts()

4    2235
1    2235
3    2232
2    2229
Name: rank, dtype: int64

In [70]:
model = LogisticRegression()
model.fit(X_train_rus, y_train_rus)

print(f'決定係数(X_train) : {model.score(X_train, y_train):.4f},\n決定係数(X_test) : {model.score(X_test, y_test):.4f}') #180個で学習したモデルで、それぞれデータにどれくらいフィットしているモデルか確認

決定係数(X_train) : 0.5570,
決定係数(X_test) : 0.5653


/Applications/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [71]:
y_pred = model.predict(X_test)

In [72]:
pred_df = pd.DataFrame({'pred':y_pred, 'actual':y_test})

In [73]:
pred_df[pred_df['pred']==1]['actual'].value_counts()

4    1800
1     694
2     584
3     480
Name: actual, dtype: int64

In [74]:
len(pred_df[pred_df['pred']==1])

3558

In [76]:
model.coef_

array([[-2.10573057e-04,  1.74832467e-03,  1.80722163e-02,
        -1.12331718e-02, -9.76447350e-02, -1.52221232e-02,
        -5.46384196e-04, -2.01594852e-04, -9.26939578e-05,
        -1.49588175e-04,  0.00000000e+00,  4.91536049e-06,
         2.15276093e-04,  6.72544050e-05,  3.43507531e-04,
         2.33100476e-04, -1.37438668e-05,  3.36353462e-04,
         6.34966361e-05, -4.82631513e-04, -4.56781420e-04,
        -5.11766026e-05,  2.13497813e-03,  1.36065089e-03,
         3.87313656e-05,  4.52926542e-04, -5.55891454e-05,
        -2.27643731e-04, -1.49631121e-04, -2.49782436e-04,
         2.05371137e-03,  5.31337050e-04, -1.69840842e-03,
         1.90859069e-05, -8.87518983e-05,  5.41528631e-05,
        -1.56563328e-05,  1.43019479e-04, -1.04637622e-04,
        -9.01603498e-06, -1.98412509e-04, -7.92862462e-05,
        -7.35049345e-04, -2.88861738e-04, -7.99391843e-04,
        -5.07359256e-04, -1.06050208e-03,  6.43729574e-05,
         1.52506747e-04, -2.13002375e-04,  0.00000000e+0

In [77]:
len(model.coef_[0]) #model.coef_[0]が回帰係数にあたる。残りの[1]~[4]は不明

198

In [78]:
len(X.columns) # 説明変数の個数の把握

198

In [79]:
coefs = pd.Series(model.coef_[0], index=X.columns).sort_values(ascending=False) #どの説明変数がどのくらい影響しているのか確認できる

In [80]:
coefs

斤量         0.018072
性_牡        0.003752
騎手_川田将雅    0.003254
騎手_ルメール    0.002135
騎手_丸山元気    0.002054
             ...   
騎手_浜中俊    -0.002017
性_牝       -0.002226
単勝        -0.011233
年齢        -0.015222
人気        -0.097645
Length: 198, dtype: float64

In [81]:
results[results['騎手']=='ルメール']['rank'].value_counts() #勝っている騎手の順位 ※一概には言えない、そもそもこの騎手がたまたまレース数が多かったからかもしれないので本当はp値などで統計的に有意性を証明する必要がある

4    270
1    155
2    120
3     75
Name: rank, dtype: int64

In [82]:
results[results['騎手']=='藤岡康太']['rank'].value_counts() #負けている騎手の順位

4    496
2     60
3     55
1     52
Name: rank, dtype: int64

In [83]:
coefs[['枠番','馬番', '斤量', '単勝', '人気', '年齢', '体重', '体重変化']].sort_values(ascending=False)

斤量      0.018072
馬番      0.001748
体重変化   -0.000202
枠番     -0.000211
体重     -0.000546
単勝     -0.011233
年齢     -0.015222
人気     -0.097645
dtype: float64

# 4. ランダムフォレストで競馬予測
前回までのデータでは時系列データとして扱っておらず、時間軸で言うと未来のデータで過去の予測を行なってしまっているという可能性もあるため  
データフレームに時間の情報も記載し時系列データとして扱う。

In [17]:
results = pd.read_pickle('results3.pickle')

In [18]:
results

,枠番,馬番,斤量,単勝,人気,年齢,体重,体重変化,rank,騎手_アヴドゥ,...,騎手_高田潤,騎手_高野和馬,騎手_鮫島克駿,騎手_鮫島良太,騎手_黒岩悠,騎手_黛弘人,騎手_Ｍ．デム,性_セ,性_牝,性_牡
201901010101,1,1,54.0,1.4,1.0,2,518,-16,1,0,...,0,0,0,0,0,0,0,0,0,1
201901010101,3,3,54.0,3.5,2.0,2,496,-8,2,0,...,0,0,0,0,0,0,0,0,0,1
201901010101,4,4,51.0,46.6,6.0,2,546,6,3,0,...,0,0,0,0,0,0,0,0,0,1
201901010101,8,9,51.0,56.8,7.0,2,458,-8,4,0,...,0,0,0,0,0,0,0,0,0,1
201901010101,5,5,54.0,140.3,9.0,2,436,0,4,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201910020812,7,12,52.0,45.7,10.0,3,414,6,4,0,...,0,0,0,0,0,0,0,0,1,0
201910020812,3,3,55.0,29.8,9.0,4,486,2,4,0,...,0,0,0,0,0,0,0,0,1,0
201910020812,5,7,55.0,29.6,8.0,4,476,6,4,0,...,0,0,0,0,0,0,0,0,1,0
201910020812,8,13,49.0,134.8,12.0,3,472,2,4,0,...,0,0,0,0,0,0,0,0,1,0


In [27]:
import requests
from bs4 import BeautifulSoup

race_id = '201901010101'
url = url = 'https://db.netkeiba.com/race/' + race_id
html = requests.get(url) # requestsでインターネットからWebページを取得
html.encoding = 'EUC-JP' # 日本語を文字化けせずに表示
soup = BeautifulSoup(html.text, 'html.parser') # HTMLを解析する

In [41]:
# 要素 = soup.find("タグ名")
# 要素のリスト = soup.find_all("タグ名")
# 要素 = soup.find(id="id名") / 要素 = soup.find(class_="class名") ※「class」はPythonの予約語でそのままでは使えないので「class_」と書く。
#soup.find('div', attrs={'class': 'data_intro'})と同じ意味で<div class="data_intro">のタグの中身を取得
soup.find(class_='data_intro').find_all('p')[0].text # textで bs4.element.Tag型 から str型 に変更

'\n\n芝右1800m\xa0/\xa0天候 : 曇\xa0/\xa0芝 : 良\xa0/\xa0発走 : 09:50\n\n\n'

In [44]:
text = soup.find(class_='data_intro').find_all('p')[0].text

In [46]:
import re

re.findall(r'\w+', text)

['芝右1800m', '天候', '曇', '芝', '良', '発走', '09', '50']

In [47]:
text2 = soup.find(class_='data_intro').find_all('p')[1].text

In [48]:
re.findall(r'\w+', text2)

['2019年7月27日', '1回札幌1日目', '2歳未勝利', '混', '指', '馬齢']

In [59]:
# race_id_list = results.index.unique()[:50] # unique()関数は重複を許さず、種類だけを取り出す
#race_id_list_50 = race_id_list[:50]

In [65]:
info

['芝右1800m',
 '天候',
 '曇',
 '芝',
 '稍重',
 '発走',
 '10',
 '25',
 '2019年8月10日',
 '1回札幌5日目',
 '2歳未勝利',
 '混',
 '指',
 '馬齢']

In [80]:
import time
from tqdm import tqdm

race_infos = {}
for race_id in tqdm(race_id_list_50):
    url = url = 'https://db.netkeiba.com/race/' + race_id
    html = requests.get(url) # requestsでインターネットからWebページを取得
    html.encoding = 'EUC-JP' # 日本語を文字化けせずに表示
    soup = BeautifulSoup(html.text, 'html.parser') # HTMLを解析する

    texts = soup.find(class_='data_intro').find_all('p')[0].text + \
        soup.find(class_='data_intro').find_all('p')[1].text # プログラムないで改行したいときは \ を使う
    info = re.findall(r'\w+', texts)
    info_dict = {}
    for text in info:
        if text in ['芝', 'ダート']: # 競馬のコースには芝とダートが存在する
            info_dict['race_type'] = text
        if '障' in text:
            info_dict['race_type'] = '障害'
        if 'm' in text:
            info_dict['cource_len'] = re.findall(r'\d+', text)[0] # findallだとリスト型で帰ってくるので[0]で文字列を取り出す
        if text in ['良', '稍重', '重', '不良']:
            info_dict['ground_state'] = text
        if text in ['曇', '晴', '雨', '小雨', '小雪', '雪']:
            info_dict['weather'] = text
        if '年' in text:
            info_dict['date'] = text
    race_infos[race_id] = info_dict
    #time.sleep(1)

100%|██████████| 50/50 [00:17<00:00,  2.82it/s]


In [81]:
pd.DataFrame(race_infos).T

,cource_len,weather,race_type,ground_state,date
201901010101,1800,曇,芝,良,2019年7月27日
201901010102,1000,曇,ダート,良,2019年7月27日
201901010103,2400,曇,ダート,良,2019年7月27日
201901010104,2000,晴,芝,良,2019年7月27日
201901010105,1500,晴,芝,良,2019年7月27日
201901010106,1700,晴,ダート,良,2019年7月27日
201901010107,1000,晴,ダート,良,2019年7月27日
201901010108,1200,晴,芝,良,2019年7月27日
201901010109,1700,晴,ダート,良,2019年7月27日
201901010110,1500,晴,芝,良,2019年7月27日


In [82]:
# for key in race_infos.keys():
#     print(key, len(race_infos[key]))

In [108]:
import time
from tqdm import tqdm as tqdm # pip install tqdm

def scrape_race_info(race_id_list, pre_race_info={}):
    race_infos = pre_race_info
    for race_id in tqdm(race_id_list):
        if race_id in race_infos.keys():
            continue
        try:
            url = url = 'https://db.netkeiba.com/race/' + race_id
            html = requests.get(url) # requestsでインターネットからWebページを取得
            html.encoding = 'EUC-JP' # 日本語を文字化けせずに表示
            soup = BeautifulSoup(html.text, 'html.parser') # HTMLを解析する
            texts = soup.find(class_='data_intro').find_all('p')[0].text + \
                soup.find(class_='data_intro').find_all('p')[1].text # プログラムないで改行したいときは \ を使う
            info = re.findall(r'\w+', texts)
            info_dict = {}
            for text in info:
                if text in ['芝', 'ダート']: # 競馬のコースには芝とダートが存在する
                    info_dict['race_type'] = text
                if '障' in text:
                    info_dict['race_type'] = '障害'
                if 'm' in text:
                    info_dict['cource_len'] = re.findall(r'\d+', text)[0] # findallだとリスト型で帰ってくるので[0]で文字列を取り出す
                if text in ['良', '稍重', '重', '不良']:
                    info_dict['ground_state'] = text
                if text in ['曇', '晴', '雨', '小雨', '小雪', '雪']:
                    info_dict['weather'] = text
                if '年' in text:
                    info_dict['date'] = text
            race_infos[race_id] = info_dict
            #time.sleep(1)
        except IndexError:
            continue
        except AttributeError:
            continue
        except:
            break
    return race_infos
    

In [105]:
race_infos = scrape_race_info(race_id_list[:50])

100%|██████████| 50/50 [00:16<00:00,  3.02it/s]


In [109]:
race_infos = scrape_race_info(race_id_list, race_infos)

100%|██████████| 4800/4800 [25:19<00:00,  3.16it/s]


In [113]:
pd.DataFrame(race_infos).T

,cource_len,weather,race_type,ground_state,date
201901010101,1800,曇,芝,良,2019年7月27日
201901010102,1000,曇,ダート,良,2019年7月27日
201901010103,2400,曇,ダート,良,2019年7月27日
201901010104,2000,晴,芝,良,2019年7月27日
201901010105,1500,晴,芝,良,2019年7月27日
...,...,...,...,...,...
201910020808,1000,曇,ダート,稍重,2019年8月18日
201910020809,1700,曇,ダート,稍重,2019年8月18日
201910020810,1800,曇,芝,良,2019年8月18日
201910020811,1200,曇,芝,良,2019年8月18日


In [114]:
pd.DataFrame(race_infos).T.to_pickle('race_infos.pickle')

In [190]:
race_infos = pd.read_pickle('race_infos.pickle')

In [191]:
race_infos

,cource_len,weather,race_type,ground_state,date
201901010101,1800,曇,芝,良,2019年7月27日
201901010102,1000,曇,ダート,良,2019年7月27日
201901010103,2400,曇,ダート,良,2019年7月27日
201901010104,2000,晴,芝,良,2019年7月27日
201901010105,1500,晴,芝,良,2019年7月27日
...,...,...,...,...,...
201910020808,1000,曇,ダート,稍重,2019年8月18日
201910020809,1700,曇,ダート,稍重,2019年8月18日
201910020810,1800,曇,芝,良,2019年8月18日
201910020811,1200,曇,芝,良,2019年8月18日


In [196]:
race_infos.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3188 entries, 201901010101 to 201910020812
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   cource_len    3188 non-null   object
 1   weather       3188 non-null   object
 2   race_type     3188 non-null   object
 3   ground_state  3188 non-null   object
 4   date          3188 non-null   object
dtypes: object(5)
memory usage: 309.4+ KB


In [201]:
#len(race_infos['cource_len'].value_counts().index)
race_infos['cource_len'].value_counts()

1800    729
1200    609
1400    420
1600    385
2000    319
1700    184
1000     81
2400     75
2200     62
1900     37
1150     35
2600     34
2100     34
1300     25
3000     22
1500     18
2500     14
2850     12
2750     12
2970     10
2890      9
2880      8
2770      6
2910      5
2860      4
3170      4
3110      4
3300      4
3330      3
3200      3
3350      2
3380      2
2300      2
3290      2
3140      2
2         1
4100      1
3930      1
3250      1
3210      1
3570      1
3900      1
4250      1
3390      1
3400      1
3100      1
Name: cource_len, dtype: int64

In [193]:
results

,枠番,馬番,斤量,単勝,人気,年齢,体重,体重変化,rank,騎手_アヴドゥ,...,騎手_高田潤,騎手_高野和馬,騎手_鮫島克駿,騎手_鮫島良太,騎手_黒岩悠,騎手_黛弘人,騎手_Ｍ．デム,性_セ,性_牝,性_牡
201901010101,1,1,54.0,1.4,1.0,2,518,-16,1,0,...,0,0,0,0,0,0,0,0,0,1
201901010101,3,3,54.0,3.5,2.0,2,496,-8,2,0,...,0,0,0,0,0,0,0,0,0,1
201901010101,4,4,51.0,46.6,6.0,2,546,6,3,0,...,0,0,0,0,0,0,0,0,0,1
201901010101,8,9,51.0,56.8,7.0,2,458,-8,4,0,...,0,0,0,0,0,0,0,0,0,1
201901010101,5,5,54.0,140.3,9.0,2,436,0,4,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201910020812,7,12,52.0,45.7,10.0,3,414,6,4,0,...,0,0,0,0,0,0,0,0,1,0
201910020812,3,3,55.0,29.8,9.0,4,486,2,4,0,...,0,0,0,0,0,0,0,0,1,0
201910020812,5,7,55.0,29.6,8.0,4,476,6,4,0,...,0,0,0,0,0,0,0,0,1,0
201910020812,8,13,49.0,134.8,12.0,3,472,2,4,0,...,0,0,0,0,0,0,0,0,1,0


In [194]:
# mergeは同じ列を探して横に結合する
# left_index, right_index を True に指定することでresultsとrace_infosのindexを同じ列として結合している
# how='inner' 内部結合（互いに存在する列のみ残し、どちらか一方しか有していない列に関しては削除する）
results_addinfo = results.merge(race_infos, left_index=True, right_index=True, how='inner')

In [195]:
results_addinfo

,枠番,馬番,斤量,単勝,人気,年齢,体重,体重変化,rank,騎手_アヴドゥ,...,騎手_黛弘人,騎手_Ｍ．デム,性_セ,性_牝,性_牡,cource_len,weather,race_type,ground_state,date
201901010101,1,1,54.0,1.4,1.0,2,518,-16,1,0,...,0,0,0,0,1,1800,曇,芝,良,2019年7月27日
201901010101,3,3,54.0,3.5,2.0,2,496,-8,2,0,...,0,0,0,0,1,1800,曇,芝,良,2019年7月27日
201901010101,4,4,51.0,46.6,6.0,2,546,6,3,0,...,0,0,0,0,1,1800,曇,芝,良,2019年7月27日
201901010101,8,9,51.0,56.8,7.0,2,458,-8,4,0,...,0,0,0,0,1,1800,曇,芝,良,2019年7月27日
201901010101,5,5,54.0,140.3,9.0,2,436,0,4,0,...,0,0,0,0,1,1800,曇,芝,良,2019年7月27日
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201910020812,7,12,52.0,45.7,10.0,3,414,6,4,0,...,0,0,0,1,0,1800,曇,芝,良,2019年8月18日
201910020812,3,3,55.0,29.8,9.0,4,486,2,4,0,...,0,0,0,1,0,1800,曇,芝,良,2019年8月18日
201910020812,5,7,55.0,29.6,8.0,4,476,6,4,0,...,0,0,0,1,0,1800,曇,芝,良,2019年8月18日
201910020812,8,13,49.0,134.8,12.0,3,472,2,4,0,...,0,0,0,1,0,1800,曇,芝,良,2019年8月18日


In [122]:
results_addinfo.isnull().sum() # それぞれの列ごとの欠損値の数

枠番              0
馬番              0
斤量              0
単勝              0
人気              0
               ..
cource_len      0
weather         0
race_type       0
ground_state    0
date            0
Length: 204, dtype: int64

In [123]:
results_addinfo.isnull().values.sum() # 欠損値の総数

0

In [130]:
results_p = preprocessing(results_addinfo)

In [155]:
results_p

,枠番,馬番,斤量,単勝,人気,年齢,体重,体重変化,rank,騎手_アヴドゥ,...,騎手_黛弘人,騎手_Ｍ．デム,性_セ,性_牝,性_牡,cource_len,weather,race_type,ground_state,date
201901010101,1,1,54.0,1.4,1.0,2,518,-16,1,0,...,0,0,0,0,1,1800,曇,芝,良,2019-07-27
201901010101,3,3,54.0,3.5,2.0,2,496,-8,2,0,...,0,0,0,0,1,1800,曇,芝,良,2019-07-27
201901010101,4,4,51.0,46.6,6.0,2,546,6,3,0,...,0,0,0,0,1,1800,曇,芝,良,2019-07-27
201901010101,8,9,51.0,56.8,7.0,2,458,-8,4,0,...,0,0,0,0,1,1800,曇,芝,良,2019-07-27
201901010101,5,5,54.0,140.3,9.0,2,436,0,4,0,...,0,0,0,0,1,1800,曇,芝,良,2019-07-27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201910020812,7,12,52.0,45.7,10.0,3,414,6,4,0,...,0,0,0,1,0,1800,曇,芝,良,2019-08-18
201910020812,3,3,55.0,29.8,9.0,4,486,2,4,0,...,0,0,0,1,0,1800,曇,芝,良,2019-08-18
201910020812,5,7,55.0,29.6,8.0,4,476,6,4,0,...,0,0,0,1,0,1800,曇,芝,良,2019-08-18
201910020812,8,13,49.0,134.8,12.0,3,472,2,4,0,...,0,0,0,1,0,1800,曇,芝,良,2019-08-18


In [141]:
results_p['date'].isnull().sum()

0

In [143]:
sorted_id_list = results_p.sort_values('date').index.unique() # dateに対して昇順でソートされ、そのindexつまりはrace_idが取り出されている

In [144]:
len(sorted_id_list)

3188

In [145]:
round(len(sorted_id_list) * 0.7)

2232

In [146]:
train_id_list = sorted_id_list[:round(len(sorted_id_list) * 0.7)]
test_id_list = sorted_id_list[round(len(sorted_id_list) * 0.7):]

In [147]:
train = results_p.loc[train_id_list] # loc：レース名のインデックスが格納されているtrain_id_listを引数とする場合 iloc ではなくラベルを指定できる loc を使用
test = results_p.loc[test_id_list]

In [148]:
len(train)

30444

In [149]:
len(train) / len(results_p)

0.7029161183071275

In [152]:
def split_data(df, test_size=0.3):
    """
        データフレームを時系列順に並べ直して、訓練データとテストデータに分割する
    """
    sorted_id_list = df.sort_values('date').index.unique()
    train_id_list = sorted_id_list[:round(len(sorted_id_list) * (1-test_size))]
    test_id_list = sorted_id_list[round(len(sorted_id_list) * (1-test_size)):]
    train = results_p.loc[train_id_list]
    test = results_p.loc[test_id_list]
    return train, test

In [153]:
train, test = split_data(results_p, 0.3)

In [154]:
len(train)

30444

In [160]:
1 if '馬名' in results_p.columns else 0

0

In [163]:
results_p.info()

<class 'pandas.core.frame.DataFrame'>
Index: 43311 entries, 201901010101 to 201910020812
Columns: 204 entries, 枠番 to date
dtypes: datetime64[ns](1), float64(3), int64(6), object(4), uint8(190)
memory usage: 14.1+ MB


In [164]:
results_d = pd.get_dummies(results_p)

In [167]:
1 if 'rank' in results_d.columns else 0

1

In [169]:
train, test = split_data(results_d, 0.3)

In [170]:
train['rank'].value_counts()

4    23748
1     2236
3     2232
2     2228
Name: rank, dtype: int64

In [188]:
# アンダーサンプリング
# 訓練データの目的変数の1,2,3,4のそれぞれの割合を均一に変更、データ数をそれぞれ約2300個の計9200個の訓練データを作成
from imblearn.under_sampling import RandomUnderSampler #pip install -U imbalanced-learn

rank_1 = train['rank'].value_counts()[1]
rank_2 = train['rank'].value_counts()[2]
rank_3 = train['rank'].value_counts()[3]


rus = RandomUnderSampler(sampling_strategy={1:rank_1, 2:rank_2, 3:rank_3, 4:rank_1}, random_state=71)

X_train = train.drop(['date', 'rank'], axis=1)
y_train = train['rank']
X_test = test.drop(['date', 'rank'], axis=1)
y_test = test['rank']

X_train_rus, y_train_rus = rus.fit_resample(X_train, y_train)

In [189]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(random_state=0)
clf.fit(X_train_rus, y_train_rus)

print(clf.score(X_train, y_train), clf.score(X_test, y_test))

ValueError: could not convert string to float: '小雨'